In [14]:
import os
os.environ["OMP_NUM_THREADS"] = "8"  # set the number of CPU threads to use for parallel regions

from pathlib import Path
import numpy as np
import pandas as pd
import time
import heaan_sdk as heaan
import math

# set key_dir_path
key_dir_path = Path('./BinaryDT/keys')

# set parameter
params = heaan.HEParameter.from_preset("FGb")

# init context and load all keys
# if set generate_keys=True, then make key
# if set generate_keys=False, then not make key. just use existing key.
context = heaan.Context(
    params,
    key_dir_path=key_dir_path,
    load_keys="all",
    generate_keys=False,
)

The currently used HEaaN library does not support GPU acceleration


In [15]:
num_slot = context.num_slots
num_slot

32768

In [16]:
log_num_slot = context.log_slots
log_num_slot

15

In [12]:
# you can see all methods in Block class
# help(heaan.Block)

### Encrypt & Decrypt
#### Save ctxt file and load

In [17]:
a = [1,2,3,4] # no need to fill 0 to make list length to num_slot
b = [0.1,0.2,0.3,0.4]

## if set encrypted = False, a will not be encrypted. just encoding list to message Block
a = heaan.Block(context,encrypted = False, data = a)
b = heaan.Block(context,encrypted = False, data = b)

## encrypt 

ctxt1 = a.encrypt()
ctxt2 = b.encrypt()

## save ciphertext to ctxt file format
ctxt1.save('/root/tutorial/python/x1.ctxt')
ctxt2.save('/root/tutorial/python/x2.ctxt')

Ciphertext(log(num_slot): 15, device: CPU, level: 12)

In [28]:
## load ctxt file

## make emtpy ctxt
empty_msg= heaan.Block(context,encrypted = False) # data : default is None
load_ctxt1 = empty_msg.encrypt()   
load_ctxt2 = empty_msg  ## after call 'empty_msg.encrypt()', then 'empty_msg' is encrypted as ciphertext

load_ctxt1 = load_ctxt1.load('/root/tutorial/python/x1.ctxt')
load_ctxt2 = load_ctxt2.load('/root/tutorial/python/x2.ctxt')

## Ciphertext print Option1. 
print_ctxt(load_ctxt1,5)
print(' ')
print_ctxt(load_ctxt2,5)
print(' ')

## Ciphertext print Option2. 
x1 = load_ctxt1.decrypt() # then load_ctxt1 is decrypted as message (it is no longer ciphertext)
x2 = load_ctxt2.decrypt()

for i in range(5):
    print('x1: ',x1[i])
    print('x2: ',x2[i])

0 (0.9999999707095197-5.234345409793269e-11j)
1 (2.000000005595655-1.2459760991605762e-08j)
2 (3.000000032556197+2.4128233938013812e-08j)
3 (3.9999999840313496-5.898575894190847e-09j)
4 (3.0732744397223015e-08+2.870428675933173e-08j)
 
0 (0.09999994851459448+4.028064524036251e-08j)
1 (0.1999999866558003+3.835441904847306e-09j)
2 (0.300000008361246+2.2082969861106234e-08j)
3 (0.40000000532473035+1.1854486555111258e-09j)
4 (-6.032135768337639e-08+1.4787881702009726e-08j)
 
x1:  (0.9999999707095197-5.234345409793269e-11j)
x2:  (0.09999994851459448+4.028064524036251e-08j)
x1:  (2.000000005595655-1.2459760991605762e-08j)
x2:  (0.1999999866558003+3.835441904847306e-09j)
x1:  (3.000000032556197+2.4128233938013812e-08j)
x2:  (0.300000008361246+2.2082969861106234e-08j)
x1:  (3.9999999840313496-5.898575894190847e-09j)
x2:  (0.40000000532473035+1.1854486555111258e-09j)
x1:  (3.0732744397223015e-08+2.870428675933173e-08j)
x2:  (-6.032135768337639e-08+1.4787881702009726e-08j)


In [20]:
def check_boot(x):
    if x.level==3:
        x.bootstrap()
    elif x.level<3:
        exit(0)
    return x

def print_ctxt(c,size):
    m = c.decrypt(inplace=False)
    for i in range(size):
        print(i,m[i])
        if (math.isnan(m[i].real)):
            print ("nan detected..stop")
            exit(0)

### Main calculation
#### Ciphertext + number

In [31]:
block = heaan.Block(context,encrypted = False, data = [0]*num_slot) ## 이렇게하면 block은 ciphertext가 아니라 message임

ctxt = block.encrypt()

## add 1 for all slots of 
ctxt = ctxt + 1

# print(ctxt)
ctxt.decrypt()
for i in range(5):
    print(ctxt[i].real)


1.0000000400842353
1.000000013139484
1.0000000267102678
0.999999970899251
1.0000000305414423


#### inverse

In [36]:
## msg * ctxt
a= [0.1,0.01,0.001,0.0001]

a_block = heaan.Block(context, data = a) 
ctxt1 = a_block.encrypt()

inverse = ctxt1.inverse(greater_than_one = False)
inverse.decrypt()
for i in range(5):
    print(inverse[i].real)

10.000001893466106
100.00010044027294
1000.0296822471282
10003.37883551541
262316.7605683955


#### add, sub, mult

#### rotate sum

In [34]:
## rotate_sum
def rotate_sum(input_ctxt):
    for i in range(int(np.log2(num_slot))):
        tmp_ctxt = input_ctxt.__lshift__(2**i)
        check_boot(tmp_ctxt)
        input_ctxt = input_ctxt + tmp_ctxt
    return input_ctxt


a= [1,2,3,4]

a_block = heaan.Block(context, data = a)
ctxt1 = a_block.encrypt()
res = rotate_sum(ctxt1)

res.decrypt()
print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])
print(res[5])
print(res[6])

(9.999997385364301+2.9296233654136295e-07j)
(9.99999740970851+3.291563313447772e-07j)
(9.999997389026195+2.8413299903565174e-07j)
(9.999997349859743+2.5802439638119684e-07j)
(9.999997462847759+3.009069589867931e-07j)
(9.999997428514376+3.0931208697553383e-07j)
(9.999997439335187+2.2211586661318477e-07j)


#### rotate
you don't need to rot_idx as block. it can be just int value.

In [40]:
a = [0.1, 0.2, 0.3, 0.4]
a = heaan.Block(context,data=a)
a.encrypt()
print(a)

rot_idx = 1
res = a.__lshift__(rot_idx) # left rotate 'a' ciphertext 'rot_idx' slots 

res.decrypt()
print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])

res2 = a.__rshift__(rot_idx) # right rotate 'a' ciphertext 'rot_idx' slots 
res2.decrypt()
print(' ')
print(res2[0])
print(res2[1])
print(res2[2])
print(res2[3])
print(res2[4])

Ciphertext(log(num_slot): 15, device: CPU, level: 12)
2
(0.2000000063268479-2.913352318509795e-09j)
(0.2999999909840392+5.586706089278126e-09j)
(0.3999999954505603-1.156802389569839e-08j)
(4.35906092725021e-09+2.871978208925836e-08j)
(-4.5824651158176474e-08-5.871980752984299e-08j)
 
(-1.935768578803946e-08-2.249148071172999e-10j)
(0.1000000059727573-1.8578693923914618e-09j)
(0.2000000065806647-2.009188390398953e-09j)
(0.29999999112003173+5.844619641396655e-09j)
(0.39999999615699633-1.2541282828238646e-08j)


In [20]:
help(heaan.Block.__neg__)
# res = self.__neg__()

Help on cython_function_or_method in module heaan_sdk.block:

__neg__(self) -> 'Block'



In [54]:
self.decrypt()
print(self[0])
print(self[1])
print(self[2])
print(self[3])
print(self[4])

(1.1000000176758764-1.2182042146624807e-08j)
(1.1999999891686257-6.9828433813540215e-09j)
(1.2999999874414672+1.0893755604136088e-08j)
(1.4000000046647083+5.507332069350436e-09j)
(-1.6586568573256016e-08+6.091451772240255e-09j)


#### negate
change the sign (ex. -2 -> +2)

In [49]:
li = [2,0,-2]+[0]*(num_slot-3)
li = heaan.Block(context, data=li)
li.encrypt()

res = 0 - li ## 0 - ciphertext is also make same result of negate method
res2 = li.__neg__()
res.decrypt()
print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])
print('')

res2.decrypt()
print(res2[0])
print(res2[1])
print(res2[2])
print(res2[3])
print(res2[4])

(-2.000000004797153+7.706221736163178e-10j)
(3.669146593665981e-08-2.5867687822515473e-09j)
(1.9999999646551068-3.1561165423301245e-08j)
(2.583266035275772e-09+1.5231461739618612e-08j)
(-3.6677957443570934e-08+6.740558198514215e-09j)

(-2.000000004797153+7.706221736163178e-10j)
(3.669146593665981e-08-2.5867687822515473e-09j)
(1.9999999646551068-3.1561165423301245e-08j)
(2.583266035275772e-09+1.5231461739618612e-08j)
(-3.6677957443570934e-08+6.740558198514215e-09j)


#### sign
- 양수면 1, 음수면 -1 나오게 하는 함수
- log_range : Log of the input range. Integer <= 38. Defaults to 0, which means that the domain of approximation is [-1, 1].

In [53]:
li = [0.2,0,-0.2]
li = heaan.Block(context, data=li)
res= li.encrypt()

res.sign(inplace = True, log_range=0)

res.decrypt()
print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])

(0.9999999996928847-2.5699293281766253e-11j)
(-0.0028869764259900794-4.2482922551060477e-10j)
(-0.9999999999756998-1.2295460578100641e-09j)
(-0.01789404673455924+2.982292119237457e-09j)
(-0.012449862036342036-8.173173899439639e-09j)


#### greater than zero
- 음수 면 0, 양수면 1, 0이면 0.5

In [52]:
li = [-0.1,0.2,0.5]
li = heaan.Block(context, data=li)
li= li.encrypt()

res= li.greater_than_zero()

res.decrypt()
print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])

(-7.426836923229985e-13+1.459841981338754e-10j)
(0.9999999997816779-2.4547904560819825e-10j)
(0.9999999998795712+1.0590633791032837e-09j)
(0.4836712262297523-2.266486432962396e-08j)
(0.49980125936617503-2.485689151228038e-09j)


In [78]:
context.min_level_for_bootstrap

3